In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import download_file
import os
from PIL import Image
from IPython.display import display
from matplotlib.patches import Circle
from astropy.wcs import WCS 
from astropy.coordinates import SkyCoord
import astropy.units as u
#import regions
from matplotlib.patches import Circle
from astropy.nddata import Cutout2D
import astropy.table as Table
from astropy.table import Column
from astroquery.simbad import Simbad
from astropy.io import ascii
from IPython.display import clear_output
from astromy_ds9 import ds9_norm
import openpyxl

In [ ]:
# Open FITS file
path = "10550_0270"
hdu_rgb = fits.open(path + "/" + path + "_ds9.fits")
hdu_i3 = fits.open(path + "/" + path + "_i3.fits")
wcs = WCS(hdu_rgb[0].header)

# Extract RGB data
RedData = hdu_rgb[1].data
GreenData = hdu_rgb[2].data
BlueData = hdu_rgb[3].data
I3Data = hdu_i3[0].data

In [ ]:
# Read scale and colormap parameters of the image from "param.txt".
# The parameter file has the following format:
#   path
#   red_low_limit
#   red_high_limit
#   green_low_limit
#   green_high_limit
#   blue_low_limit
#   blue_high_limit
#   i3_low_limit
#   i3_high_limit
#   red_contrast
#   red_bias
#   green_contrast
#   green_bias
#   blue_contrast
#   blue_bias
#   i3_contrast
#   i3_bias

param_file = open("param.txt")
lst = param_file.read().split()
# Find index of the line which contains the name of the region we're analyzing.
i = lst.index(path)
# Create a sub-list with the image parameters and convert all the strings to floats.
new_lst = (float(j) for j in lst[i+1:i+17])

red_low_limit, red_high_limit, green_low_limit, green_high_limit, blue_low_limit, blue_high_limit, i3_low_limit, i3_high_limit, red_contrast, red_bias, green_contrast, green_bias, blue_contrast, blue_bias, i3_contrast, i3_bias = new_lst

In [ ]:
def update_image(RedData, GreenData, BlueData, I3Data):
    # Apply scaling to each channel to emphasize peaks and reduce noise
    Rednorm = ds9_norm(vmin=red_low_limit, vmax=red_high_limit, bias=red_bias, contrast=red_contrast, stretch='sqrt')(RedData)
    Greennorm = ds9_norm(vmin=green_low_limit, vmax=green_high_limit, bias=green_bias, contrast=green_contrast, stretch='sqrt')(GreenData)
    Bluenorm = ds9_norm(vmin=blue_low_limit, vmax=blue_high_limit, bias=blue_bias, contrast=blue_contrast, stretch='sqrt')(BlueData)
    I3norm = ds9_norm(vmin=i3_low_limit, vmax=i3_high_limit, bias=i3_bias, contrast=i3_contrast, stretch='sqrt')(I3Data)

    # Stack the normalized channels
    rgb_image = np.dstack((Rednorm, Greennorm, Bluenorm))

    # Display the RGB image
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_image, origin='lower')
    plt.show()
    
    return Rednorm, Greennorm, Bluenorm, I3norm


new_RedScaled, new_GreenScaled, new_BlueScaled, new_I3Scaled = update_image(RedData, GreenData, BlueData, I3Data)

In [ ]:
#Funcion myRegion
def myRegion(x_coord, y_coord, image, wcs, filtro, half_size=100):
    
    # Ensure the provided coordinates are in pixel units
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region
    center_x = half_size
    center_y = half_size

    # Create a figure and axis
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    print(plt.rcParams["figure.dpi"])
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the region using WCS information from the red channel
    cutout = Cutout2D(image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Display the region extracted with WCS
    ax.imshow(cutout.data, origin='lower')

    # Disable axis labels and ticks
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Display the plot
    print(filtro)
    # plt.show()

    return fig
    

In [ ]:
  #Funcion overlayFilters (funcion para hacer un overlay de los 3 filtro RGB)
def overlayFilters(x_coord, y_coord, red_image, green_image, blue_image, wcs, filtro, half_size=100,):
    # Ensure the provided coordinates are in pixel units
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region
    center_x = half_size
    center_y = half_size

    # Create a figure and axis
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    print(plt.rcParams["figure.dpi"])
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the regions using WCS information from the different filters
    red_cutout = Cutout2D(red_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    green_cutout = Cutout2D(green_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    blue_cutout = Cutout2D(blue_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Combine the filters into a composite RGB image
    composite_image = np.dstack((red_cutout.data, green_cutout.data, blue_cutout.data))

    # Display the composite image with WCS
    ax.imshow(composite_image, origin='lower')

    # Disable axis labels and ticks
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Display the plot
    print(filtro)
    #plt.show()
    
    return fig

In [ ]:
# Define the filename for the table
filename = path + '/region_info_' + path + '.csv'

# Reads region file in physical coordinates
region_file_physical = path + '/fisicas_' + path + '.reg'
# Reads region file in galactic coordinates
region_file_galactic = path + '/galacticas_' + path + '.reg'  

In [ ]:
### Verify if table exists.
try:
    data_table = ascii.read(filename)
except FileNotFoundError:
    data_table = Table(names=('x_coord', 'y_coord', 'shape_label', 'description'), 
                       dtype=('f8', 'f8', 'S20', 'S100'))
    
####################################################################################################################

with open(region_file_physical, 'r') as f_physical:
    lines_physical = f_physical.readlines()

with open(region_file_galactic, 'r') as f_galactic:
    lines_galactic = f_galactic.readlines()

# Initialize Excel file.
wb = openpyxl.Workbook()
ws = wb.worksheets[0]
count = 1
tile = 2

ws["A1"] = "Number"
ws["B1"] = "Gal. X"
ws["C1"] = "Gal. Y"
ws["D1"] = "Galaxy"
ws["E1"] = "RGB Img"
ws["I1"] = "R Filter"
ws["M1"] = "I3 Filter"
ws["Q1"] = "G Filter"
ws["U1"] = "B Filter"

# Iterate over each line in the physical coordinate file
for line_physical, line_galactic in zip(lines_physical, lines_galactic):
    # Remove leading/trailing whitespace and skip empty lines
    line_physical = line_physical.strip()
    line_galactic = line_galactic.strip()
    
    # Extract the physical and Galactic coordinates
    physical_parts = line_physical.split()
    galactic_parts = line_galactic.split()
    
    physical_x_str, physical_y_str = physical_parts[0], physical_parts[1]
    x = float(physical_x_str)
    y = float(physical_y_str)

    galactic_x_str, galactic_y_str = galactic_parts[0], galactic_parts[1]
    galactic_x = float(galactic_x_str)
    galactic_y = float(galactic_y_str)
    
    # Check if the data already exists in the table
    if any((data_table['x_coord'] == galactic_x) & (data_table['y_coord'] == galactic_y)):
        # If data exists, skip
        print("Data already exists in the table for Galactic coordinates:", galactic_x_str, galactic_y_str)
        continue
      
    else:

        # Print the Galactic coordinates
        print("Region Galactic:")
        print(galactic_x_str, galactic_y_str)
        print()

        # Functions to display images
        figure = overlayFilters(x, y, new_RedScaled, new_GreenScaled, new_BlueScaled, wcs, "RGB", half_size=50)

        # Save figure into image.
        rgb_name = "cuts/rgb_plot" + str(count) + ".png"
        r_name = "cuts/r_plot" + str(count) + ".png"
        g_name = "cuts/g_plot" + str(count) + ".png"
        b_name = "cuts/b_plot" + str(count) + ".png"
        i3_name = "cuts/i3_plot" + str(count) + ".png"

        plt.savefig(rgb_name)
        fig1 = myRegion(x, y, new_RedScaled, wcs, "Rojo-8 micras", half_size=50)
        plt.savefig(r_name)
        fig2 = myRegion(x, y, new_GreenScaled, wcs, "Verde-4.6 micras", half_size=50)
        plt.savefig(g_name)
        fig3 = myRegion(x, y, new_BlueScaled, wcs, "Azul-3.5 micras", half_size=50)
        plt.savefig(b_name)
        fig4 = myRegion(x, y, new_I3Scaled, wcs, "5.8 micras", half_size=50)
        plt.savefig(i3_name)

        # Write figure and data into Excel spreadsheet.
        ws["A" + str(tile)] = count
        ws["B" + str(tile)] = galactic_x
        ws["C" + str(tile)] = galactic_y
        ws["D" + str(tile)] = "Yes"

        rgb_img = openpyxl.drawing.image.Image(rgb_name)
        r_img = openpyxl.drawing.image.Image(r_name)
        g_img = openpyxl.drawing.image.Image(g_name)
        b_img = openpyxl.drawing.image.Image(b_name)
        i3_img = openpyxl.drawing.image.Image(i3_name)

        ws.add_image(rgb_img, "E" + str(tile))
        ws.add_image(r_img, "I" + str(tile))
        ws.add_image(i3_img, "M" + str(tile))
        ws.add_image(g_img, "Q" + str(tile))
        ws.add_image(b_img, "U" + str(tile))
        wb.save("output.xlsx")

        # Close the image.
        plt.close(figure)
        count += 1
        tile += 13

        # Clear the output cell in Jupyter Notebook
        clear_output(wait=False)